Copyright 2022 AudioT.  All rights reserved.
This code is proprietary to AudioT, INC and provided to OMSA students under the conditions of the AudioT NDA terms. 

# Goal

The purpose of this notebook is to show what log mel energy features (and their deltas and delta-
deltas) look like, and to demonstrate how different parameter settings affect them.  Specifically,
we'll be looking at a recording from early in a flock with some distress calling and how those calls
show up in the features.

# Imports / environment setup

**NOTE:** You will need to install the ipykernel package into your virtual environment (if you have not already done so) to be able to run the code in this notebook.  To do so, open an Anaconda prompt, activate the virtual environment you will be using, and run `conda install ipykernel`.


In [ ]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import scipy as sp
import scipy.signal
import matplotlib as mpl
import matplotlib.pyplot as plt

# Show matplotlib plots inline in Jupyter notebooks without having to call show()
%matplotlib inline

import IPython.display as ipd
import librosa

from audiot.audio_signal import AudioSignal
from audiot.audio_features import AudioFeatures, calc_log_mel_energy_features

# Parameters

In [ ]:
figure_size = (30, 4)
project_folder = Path("").absolute().parents[0]
file_path = project_folder / "test_data" / "TRF0_mic14_2020-12-17_01.20.00.flac"

# Start and end time for the segment we will be looking at
segment_of_interest_start_time = 5
segment_of_interest_end_time = 10

# Short time fourier transform (STFT) parameters -- for computing spectrograms
fft_n_samples_per_window = 256
fft_n_overlap = fft_n_samples_per_window / 2
fft_nfft = 2 ** np.ceil(np.log2(fft_n_samples_per_window))

# Compute spectrogram
Parameters:
- **fs:** The sampling rate for the audio signal.
- **nperseg:** The number of samples in each window over which the Fourier transform is computed for
    the short time Fourier transform (STFT).
- **noverlap:** The number of samples that overlap for adjacent windows in the STFT.
- **nfft:** Number of samples used under the hood for the fast Fourier transform (FFT).  Generally, 
    it's best to set this to the next power of two larger than or equal to the window size 
    (nperseg).

In [ ]:
audio_signal = AudioSignal.from_file(file_path)
(spectrogram_frequencies, spectrogram_times, spectrogram) = sp.signal.spectrogram(
    audio_signal.signal[:, 0],
    fs=audio_signal.sample_rate,
    nperseg=fft_n_samples_per_window,
    noverlap=fft_n_overlap,
    nfft=fft_nfft,
)
spectrogram = np.log(spectrogram)


# Plot spectrogram

For this recording, a fan is on at the beginning of the recording and switches off around 6 seconds 
in.  Right after that fan switches off, another fan comes on between 7 and 8 seconds that is louder
(in particular, it creates a lot more noise in the higher frequency ranges).  That second fan 
switches off around 35 seconds, and there is little to no fan noise for the rest of the file after
that.

Distress calls from one bird in particular (that was probably close to the mic) can be heard very
clearly for about the first two thirds of the file.  There is also much quieter background chirping
babble throughout the file from birds that are probably farther away from the mic.

I first plot the spectrogram for the entire file with the spot I intend to zoom in on marked.  Then
I plot the spectrogram zoomed in on that specific segment of interest.  The increased zoom makes it
much easier to see the shape of the vocalizations.  The zoomed segment also includes the spot where
the first fan switches off and the the second fan switches on.

In [ ]:
# Plot the spectrogram of the file
fig = plt.figure(figsize=figure_size)
axes = fig.add_subplot(1, 1, 1)
axes.imshow(
    spectrogram,
    aspect="auto",
    origin="lower",
    extent=[
        spectrogram_times[0],
        spectrogram_times[-1],
        spectrogram_frequencies[0],
        spectrogram_frequencies[-1],
    ],
)
axes.vlines([segment_of_interest_start_time, segment_of_interest_end_time], 0, 8000, color="r")

# Zoom in on some vocalizations before and after a fan turns on
fig = plt.figure(figsize=figure_size)
axes = fig.add_subplot(1, 1, 1)
axes.imshow(
    spectrogram,
    aspect="auto",
    origin="lower",
    extent=[
        spectrogram_times[0],
        spectrogram_times[-1],
        spectrogram_frequencies[0],
        spectrogram_frequencies[-1],
    ],
)
axes.set_xlim(segment_of_interest_start_time, segment_of_interest_end_time)

# Add a control to listen to the segment we zoomed in on
ipd.Audio(
    audio_signal.signal[
        int(segment_of_interest_start_time * audio_signal.sample_rate) : int(
            segment_of_interest_end_time * audio_signal.sample_rate
        ),
        -1,
    ],
    rate=int(audio_signal.sample_rate),
)


# Log mel energy features

Define a function and a wrapper for that function to make it easy to plot features zoomed in on the
segment of interest.

In [ ]:
# Define a function to make plotting easier
def plot_features_xlim(features, xlim):
    fig = plt.figure(figsize=figure_size)
    axes = fig.add_subplot(1, 1, 1)
    axes_image = axes.imshow(
        features.features.T,
        aspect="auto",
        origin="lower",
        extent=[
            features.frame_start_times[0],
            features.frame_start_times[-1] + features.frame_duration,
            0,
            features.n_features,
        ],
    )
    axes.set_xlim(xlim[0], xlim[-1])
    return fig, axes, axes_image


# Wrap the above function so we don't have to pass the start/end time of the segment of interest.
plot_features = lambda features: plot_features_xlim(
    features, (segment_of_interest_start_time, segment_of_interest_end_time)
)

Show what the default log mel energy features look like if you don't pass any parameters.

In [ ]:
log_mel_energies = calc_log_mel_energy_features(audio_signal)
plot_features(log_mel_energies)
plt.title("Log mel energies: default parameters")


Notice that although some of the distress calls have different shapes in the spectrogram we plotted
before, you can't really distinguish the difference in shape in the log mel energy plot above.  If
this shape is important to what you want to do, you could consider modifying the parameters to try
to represent that better.  The three main parameters you might consider adjusting are these:
- `n_mels` -- This represents the number of frequency bands in the mel filter bank, which equates to
    the resolution along the frequency axis (the number of rows) in the plot above.  Increasing it will
    give us better vertical resolution, but comes at the cost of increasing the dimensionality of your
    features.  Adjacent frequency bands that are narrower (higher resolution) will tend to be more
    strongly correlated with each other as well.  Defaults to 13.
- `min_frequency` -- The triangular filters in the mel filter bank are distributed between a minimum
    and a maximum frequency.  Only the energy / sounds that fall within this frequency range will be
    reflected in the features, while sounds below the min frequency and above the max frequency will be
    ignored.  This specifies the minimum frequency for that range.  Defaults to 100Hz.
- `max_frequency` -- Same as above, but this specifies the maximum frequency for the range over
    which the triangular mel filter bank is distributed.  Defaults to 8000Hz.

To try to capture the shape of the distress calls better, we could try increasing the resolution by
bumping up the `n_mels` parameter:

In [ ]:
min_frequency = 100  # Same as default value
max_frequency = 8000 # Same as default value
n_mels = 40          # Increased from the default of 13
log_mel_energies = calc_log_mel_energy_features(
    audio_signal, min_frequency=min_frequency, max_frequency=max_frequency, n_mels=n_mels
)
plot_features(log_mel_energies)
plt.title(f"Log mel energies: {min_frequency}-{max_frequency}Hz, n_mels={n_mels}")

We can indeed see the shape of the individual distress calls better above, but we've also increased
the dimensionality of our features quite a lot.  However, many of the features (e.g. the ones
between 0 and 20 on the y-axis) are probably not very useful in detecting the distress calls because
they fall outside the frequency range over which distress calls occur.  If we look at the original
spectrogram, the distress calls at this age of the chickens tend to occur in the 2000-4500Hz
frequency range.  We could potentially select out the relevant features from above (e.g. between 20
and 35 on the y-axis), or we can just adjust the parameters to only compute features over that range
(lowering `n_mels` as appropriate for the smaller frequency range):

In [ ]:
min_frequency = 2000
max_frequency = 4500
n_mels = 15
log_mel_energies = calc_log_mel_energy_features(
    audio_signal, min_frequency=min_frequency, max_frequency=max_frequency, n_mels=n_mels
)
plot_features(log_mel_energies)
plt.title(f"Log mel energies: {min_frequency}-{max_frequency}Hz, n_mels={n_mels}")

The above looks pretty good for representing the different shapes of the distress calls without
having lots of extra, unneeded feature dimensions.  Note that 2000-4500Hz seems like a good range
for distress calls in the first week, but this range will probably need to be lowered for older
chickens (e.g. you might need to drop it by 500Hz for week two, and maybe more later).

In selecting parameter values, always keep in mind what goal you're trying to achieve.  For example,
if you're just trying to detect distress calls and don't really care about slight differences in
their shape, then you might consider going back to a lower `n_mels` value to lower the
dimensionality and make all the distress calls appear more similar.  This should be fine as long as
the lower resolution doesn't make it hard to distinguish between distress calls and other souds in
the environment:

In [ ]:
min_frequency = 2000
max_frequency = 4500
n_mels = 5
log_mel_energies = calc_log_mel_energy_features(
    audio_signal, min_frequency=min_frequency, max_frequency=max_frequency, n_mels=n_mels
)
plot_features(log_mel_energies)
plt.title(f"Log mel energies: {min_frequency}-{max_frequency}Hz, n_mels={n_mels}")

# Delta and delta-delta features

I've previously mentioned how delta and delta-delta features are one way to incorporate more
information about how the features are changing over time.  Here, I want to show you what those
features look like as well as another very nice property of them.

In [ ]:
min_frequency = 2000
max_frequency = 4500
n_mels = 5
log_mel_energies = calc_log_mel_energy_features(
    audio_signal, min_frequency=min_frequency, max_frequency=max_frequency, n_mels=n_mels
)
deltas = log_mel_energies.deltas()
delta_deltas = deltas.deltas()
features = AudioFeatures.stack(log_mel_energies, deltas, delta_deltas)
fig, axes, axes_image = plot_features(features)
axes.hlines([n_mels, 2*n_mels], 0, audio_signal.duration, color='r')
fig.colorbar(axes_image)
plt.title("Log mel energies (bottom 5 rows), deltas (middle 5 rows), and delta-deltas (top 5 rows)")

I drew horizontal red lines to visually separate the original log mel energies (on bottom) from
their deltas (middle) and delta-deltas (top).

The deltas and delta-deltas are just estimates of the first and second time derivatives of the
original log mel energy features.  So notice how for each chirp, the deltas first go positive and
then go negative.  This represents the upward slope of the log mel energies as the chirp sound
arrives and the downward slope of those energies as the chirp sound ends.  Similarly, you see a
positive-negative-positive pattern for each chirp in the delta-deltas.  This represents that each
log mel energy first curves up as the energy from the chirp first starts arriving in that frequency
band, then curves downward at the peak of the chirp sound, then curves up again as it flattens back
out after the end of the chirp.

You may also notice that the log mel energies along the bottom tend to be negative when there's not
much going on.  This does not mean that there is some kind of "negative" sound energy, but rather is
just an artifact of applying a log scaling to the original energy in each frequency band.  If the
energy in a given band ends up with a value less than 1.0, then taking the log of that energy value
will result in a negative number.

**A very nice property about the deltas and delta-deltas is their invariance to constant background
noise (taking a derivative causes constant terms to drop out).**  Notice that the noise from the 
two fans is visible in the log mel energies at the bottom of the plot as a general increase in the
background brightness / energy between the chirps.  The slightly darker region between about 6.5-7.0
seconds is the gap between the first fan turning off, and the second (louder) fan turning on.  Any
models that do classification based on these log mel energies will have to try to learn how to 
properly classify things both with and without that extra background energy from the fans.

However, if you look at the deltas (middle) and delta-deltas (top), you'll notice that there is no
visible difference between when the fans are on and off.  This is because the fan noise doesn't
cause rapid changes in the amount of energy present in the frequency bands, and thus does not have
much effect on the slope of how those energies are changing over time.  Thus, for tasks where the
fan noise it not important, it may be worth considering excluding the original log mel energies and 
only useing their deltas and/or delta-deltas as the features you train and run your model on.

Here's what the above delta features look like for the entire file, compared to the spectrogram:

In [ ]:
min_frequency = 2000
max_frequency = 4500
n_mels = 5
log_mel_energies = calc_log_mel_energy_features(
    audio_signal, min_frequency=min_frequency, max_frequency=max_frequency, n_mels=n_mels
)
deltas = log_mel_energies.deltas()
fig, axes, axes_image = plot_features_xlim(deltas, (0, 60))
plt.title("Log mel energies (bottom 5 rows), deltas (middle 5 rows), and delta-deltas (top 5 rows)")

fig = plt.figure(figsize=figure_size)
axes = fig.add_subplot(1, 1, 1)
axes.imshow(
    spectrogram,
    aspect="auto",
    origin="lower",
    extent=[
        spectrogram_times[0],
        spectrogram_times[-1],
        spectrogram_frequencies[0],
        spectrogram_frequencies[-1],
    ],
)

By default, the function to compute deltas looks at the current feature frame as well as the two
frames before it and the two frames after it to estimate the derivative (the `n_samples_per_side`
parameter defaults to 2).  This is why the effects of a single chirp look a little more spread out
along the time axis for the deltas than it is for the original log mel energies.  Similarly, the
effects of a single chirp are spread out even further for the delta-deltas due to applying the
function a second time.  The `n_samples_per_side` parameter can be used to adjust how big the delta
computation window is as appropriate for your application.  Here are examples of what it looks like
if you decrease or increase it for this file:

In [ ]:
n_samples_per_side = 1
min_frequency = 2000
max_frequency = 4500
n_mels = 5
log_mel_energies = calc_log_mel_energy_features(
    audio_signal, min_frequency=min_frequency, max_frequency=max_frequency, n_mels=n_mels
)
deltas = log_mel_energies.deltas(n_samples_per_side=n_samples_per_side)
delta_deltas = deltas.deltas(n_samples_per_side=n_samples_per_side)
features = AudioFeatures.stack(log_mel_energies, deltas, delta_deltas)
fig, axes, axes_image = plot_features(features)
axes.hlines([n_mels, 2*n_mels], 0, audio_signal.duration, color='r')
fig.colorbar(axes_image)
plt.title(f"n_samples_per_side={n_samples_per_side}")

n_samples_per_side = 3
deltas = log_mel_energies.deltas(n_samples_per_side=n_samples_per_side)
delta_deltas = deltas.deltas(n_samples_per_side=n_samples_per_side)
features = AudioFeatures.stack(log_mel_energies, deltas, delta_deltas)
fig, axes, axes_image = plot_features(features)
axes.hlines([n_mels, 2*n_mels], 0, audio_signal.duration, color='r')
fig.colorbar(axes_image)
plt.title(f"n_samples_per_side={n_samples_per_side}")

Note that the spreading effect is lessened for smaller values of `n_samples_per_side` and increased
for larger values.

# Other features

Currently, log mel energies (and variations on them, like their deltas and delta-deltas) are
probably the most commonly used features for sound event detection.  This is likely because they're
much like a spectrogram (that's been down-sampled to a more reasonable dimensionality) that can
visualize the frequencies present in any sound---and modern ML techniques are able to deal with
image-like data like that pretty well.  Other audio features have also been used in the past, and
there are various libraries out there that can compute some of them (including librosa).

Below is an example of the spectral centroid feature from librosa.  This feature effectively
computes and average location of all the energy along the frequency axis (the y axis in these
plots).  Note that while it looks really nice for detecting the distress calls at the very beginning
of the file when there's low-pitched fan noise in the background, it doesn't work as well in other
parts of the recording.  The second fan creates more energy in the higher frequencies, thus pulling
the centroid up closer to where it tends to be for the distress calls.  Later when there is almost 
no fan noise, the energy from the background chirping becomes the driver of where the spectral 
centroid ends up being located.  So while other audio features like this can be useful in certain
situations, it's important to consider what types of sounds could throw them off in undesirable
ways and how likely those types of sounds are to occur in production environments.

In [ ]:
spectral_centroid = librosa.feature.spectral_centroid(y=audio_signal.signal[:,0], sr=audio_signal.sample_rate)
spectral_centroid_times = librosa.times_like(spectral_centroid, sr=audio_signal.sample_rate)

fig = plt.figure(figsize=figure_size)
axes = fig.add_subplot(1, 1, 1)
axes.imshow(
    spectrogram,
    aspect="auto",
    origin="lower",
    extent=[
        spectrogram_times[0],
        spectrogram_times[-1],
        spectrogram_frequencies[0],
        spectrogram_frequencies[-1],
    ],
)
axes.plot(spectral_centroid_times, spectral_centroid[0,:])

# Zoom in on some vocalizations before and after a fan turns on
fig = plt.figure(figsize=figure_size)
axes = fig.add_subplot(1, 1, 1)
axes.imshow(
    spectrogram,
    aspect="auto",
    origin="lower",
    extent=[
        spectrogram_times[0],
        spectrogram_times[-1],
        spectrogram_frequencies[0],
        spectrogram_frequencies[-1],
    ],
)
axes.set_xlim(segment_of_interest_start_time, segment_of_interest_end_time)
axes.plot(spectral_centroid_times, spectral_centroid[0,:])